In [ ]:
# FACIAL RECOGNITION USING SIAMESE NETWORKS

In [2]:
import os
import numpy 
import random
import cv2 #computer vision library
import matplotlib.pyplot as plt

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D, Dense, Input, Flatten

In [4]:
import tensorflow as tf

In [4]:
# Avoid Out of memory error by setting GPU memory consumption growth
gpus = tf.config.experimental.list_physical_devices('gpu')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
POS_PATH = os.path.join('data', 'positive') #for positive verification images
NEG_PATH = os.path.join('data', 'negative') #for negative verification images
ANC_PATH = os.path.join('data', 'anchor') #for base images

In [6]:
# Making Directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [7]:
# Downloading Negative Dataset
# uncompress the tar file
!tar -xf lfw.tgz

In [8]:
# Move lfw images(extracted tar file) to negative folder
for directories in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw',directories)):
        old = os.path.join('lfw', directories, file)
        new = os.path.join(NEG_PATH, file)
        os.replace(old, new)

In [124]:
# # checking whether the images are stored in the negative directory
# for file in os.listdir(os.path.join('data', 'negative')):
# #     print(os.path.join('data', 'negative', file))

In [10]:
# Collecting Positive and Anchor Dataset

In [11]:
# each image needs a unique id, so let's import uuid (universally unique identifiers)
!pip install uuid
import uuid

In [12]:
'{}.jpg'.format(uuid.uuid1()) #creating unique id's for each image

'be200711-c43f-11ee-a6be-95c89532f882.jpg'

In [13]:
'{}.jpg'.format(uuid.uuid1())

'be21ed7d-c43f-11ee-b690-95c89532f882.jpg'

In [14]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1())) # getting stored into data\\anchor folder (our base images folder)

'data\\anchor\\be23c984-c43f-11ee-b392-95c89532f882.jpg'

In [61]:
# Establishing a new connection to the webcam to collect anchor values
cap = cv2.VideoCapture(0) # 3-> webcam value points to the device
while cap.isOpened():
    ret, frame = cap.read()
    
    frame = frame[120:120+250, 200:200+250, : ] #actually, the image size should be (250, 250, 3) (height, width, no.of.channels)
    
    #Collect both Anchors (base image) and Positives
    # for Anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    #for Positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgpos = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgpos, frame)
    
    # show image back to screen
    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [16]:
# cv2.waitKey??

In [17]:
# plt.imshow(frame)

In [18]:
# frame.shape

In [19]:
# plt.imshow(frame[120:120+250, 200:200+250, :])

In [20]:
# frame[120:120+250, 200:200+250, :].shape

In [6]:
# Let's get the images from Directories
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(100)
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(100)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(100)

In [7]:
len(anchor)

100

In [8]:
len(positive)

100

In [9]:
len(negative)

100

In [10]:
# Preprocessing the Data
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (105, 105))
    img = img / 255.0 # normalizing the image
    return img

In [11]:
# Important Step
# Creating Labelled Dataset
# (positive, anchor) -> 1 / (negative, anchor) -> 0
positives = tf.data.Dataset.zip(anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))))
negatives = tf.data.Dataset.zip(anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor))))
data = positives.concatenate(negatives)

In [12]:
len(data)

200

In [13]:
samples = data.as_numpy_iterator()

In [14]:
samples.next()

(b'data\\anchor\\068935a7-c44c-11ee-9086-95c89532f882.jpg',
 b'data\\positive\\7c7d1795-c44c-11ee-b794-95c89532f882.jpg',
 1.0)

In [15]:
# Building Training and Testing partition
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [16]:
# Building DataLoader Pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [17]:
data

<_ShuffleDataset element_spec=(TensorSpec(shape=(105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [18]:
# samples = data.as_numpy_iterator()

In [19]:
# len(samples.next())

In [20]:
# samp = samples.next()

In [21]:
# plt.imshow(samp[1])

In [22]:
#plt.imshow(samp[0]) # mine anchor image

In [23]:
# samp[2] #negative

In [24]:
# Partitioning
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [25]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [103]:
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [104]:
test_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 105, 105, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

In [71]:
# Creating Convolution Layers
def embedding():
    inp = Input(shape = (105, 105, 3), name = 'input_image')
    cnn1 = Conv2D(64, (10,10), activation = 'relu')(inp)
    mp1 = MaxPooling2D(64, (2,2), padding = 'same')(cnn1)
    cnn2 = Conv2D(128, (7,7), activation = 'relu')(mp1)
    mp2 = MaxPooling2D(64, (2,2), padding = 'same')(cnn2)
    cnn3 = Conv2D(128, (4,4), activation = 'relu')(mp2)
    mp3 = MaxPooling2D(64, (2,2), padding = 'same')(cnn3)
    cnn4 = Conv2D(256, (4,4), activation = 'relu')(mp3)
    f1 = Flatten()(cnn4)
    d1 = Dense(4096, activation = 'sigmoid')(f1)
    
    return Model(inputs = [inp], outputs = [d1], name = "embedding")

In [72]:
model = embedding()

In [73]:
model.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 21, 21, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 18, 18, 128)       26

In [74]:
# Creating Distance Layer
class L1Dist(Layer):
    def __init__(self, *kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [75]:
# siamese_layer = L1Dist()
# input_image = Input(name = 'input_img', shape = (105, 105, 3))
# validation_image = Input(name = 'validation_img', shape = (105, 105, 3))
# distances = siamese_layer(model(input_image), model(validation_image))

In [76]:
# output = Dense(1, activation = 'sigmoid')(distances)

In [77]:
# siamnet = Model(inputs = [input_image, validation_image], outputs = output, name = 'SiameseNetwork')

In [78]:
# siamnet.summary()

In [79]:
def siamese_model():
    input_image = Input(name = 'input_img', shape = (105, 105, 3))
    validation_image = Input(name = 'validation_img', shape = (105, 105, 3))
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(model(input_image), model(validation_image))
    
    # final classification layer
    output = Dense(1, activation = 'sigmoid')(distances)
    
    return Model(inputs = [input_image, validation_image], outputs = output, name = 'SiameseNetwork')

In [80]:
simodel = siamese_model()

In [81]:
simodel.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_img (InputLayer)      [(None, 105, 105, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 105, 105, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_img[0][0]',           
                                                          8          'validation_img[0][0]']      
                                                                                     

In [82]:
# loss and optimizer
binary_loss = tf.losses.BinaryCrossentropy()
optimizers = tf.keras.optimizers.Adam(1e-4)

In [90]:
@tf.function
# creating train function
def train_step(batch):
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]
        
        # defining the model
        yhat = simodel(X, training = True)
        # loss
        loss = binary_loss(y, yhat)
    print(loss)
    
    # finding out the gradients
    grad = tape.gradient(loss, simodel.trainable_variables)
    # calculating weights and applying it to siamese_model
    optimizers.apply_gradients(zip(grad, simodel.trainable_variables))
    
    return loss

In [91]:
def train(data, epochs):
    for epoch in range(1, epochs+1):
        print('\n Epoch {}/{}'.format(epoch, epochs)) # printing epoch 1/10
        progbar = tf.keras.utils.Progbar(len(data))
        
        #loop through each batch
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)

In [92]:
train(train_data, 50)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
9/9 [==============================] - 29s 3s/step

 Epoch 2/50
9/9 [==============================] - 25s 3s/step

 Epoch 3/50
9/9 [==============================] - 25s 3s/step

 Epoch 4/50
9/9 [==============================] - 25s 3s/step

 Epoch 5/50
9/9 [==============================] - 26s 3s/step

 Epoch 6/50
9/9 [==============================] - 26s 3s/step

 Epoch 7/50
9/9 [==============================] - 29s 3s/step

 Epoch 8/50
9/9 [==============================] - 31s 3s/step

 Epoch 9/50
9/9 [==============================] - 31s 3s/step

 Epoch 10/50
9/9 [==============================] - 28s 3s/step

 Epoch 11/50
9/9 [==============================] - 28s 3s/step

 Epoch 12/50
9/9 [==============================] - 27s 3s/step

 Epoch 13/50
9/9 [==============================] - 27s 3s/step

 Epoch 14

In [ ]:
# Evaluation Metrics -> Precision and Recall to check our Model's performance

In [114]:
from tensorflow.keras.metrics import Precision, Recall

In [106]:
test_input, test_val, ytest = test_data.as_numpy_iterator().next()

In [107]:
predictions = simodel.predict([test_input, test_val])
predictions

1/1 [==============================] - 1s 775ms/step


array([[9.9998379e-01],
       [1.0000000e+00],
       [9.9472737e-01],
       [1.6050297e-12],
       [9.9692780e-01],
       [9.9999863e-01],
       [9.9999058e-01],
       [9.9996865e-01],
       [9.9855810e-01],
       [6.3393891e-08],
       [7.4985497e-11],
       [1.0000000e+00],
       [1.5638598e-11],
       [4.6658052e-11],
       [9.9935991e-01],
       [1.5963143e-07]], dtype=float32)

In [108]:
[1 if predict>0.5 else 0 for predict in predictions]

[1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0]

In [111]:
n = Recall()
n.update_state(ytest, predictions)
n.result().numpy()

1.0

In [112]:
m = Precision()
m.update_state(ytest, predictions)
m.result().numpy()

1.0

In [116]:
# Visualizing results -> ya, it's worked
# plt.figure(figsize = (18,8))
# plt.subplot(1, 2, 1)
# plt.imshow(test_input[0])
# plt.subplot(1, 2, 2)
# plt.imshow(test_val[0])

In [120]:
# plt.figure(figsize = (18,8))
# plt.subplot(1, 2, 1)
# plt.imshow(test_input[3])
# plt.subplot(1, 2, 2)
# plt.imshow(test_val[3])